In [29]:
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader
import numpy as np

# Загружаем датасет с рейтингами фильмов.
ratings = pd.read_csv("ratings.csv").head(1000)

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,10,11,...,91529,91658,99114,106782,109487,112552,114060,115713,122882,131724
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,2.5,3.5,5.0,3.0,4.0,2.0,3.5,5.0,5.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,4.0,5.0,3.0,5.0,4.0,4.0,3.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
class MySlopeOne:
    def __init__(self):
        # Инициализация хранилищ для отклонений и количества оценок между парами элементов.
        self.deviations = {}  # Словарь для хранения отклонений между парами фильмов
        self.counts = {}  # Словарь для хранения количества совместных оценок для каждой пары фильмов

    def fit(self, ratings):
        # Получение уникальных идентификаторов пользователей и фильмов из датасета.
        users = ratings['userId'].unique()
        items = ratings['movieId'].unique()

        # Преобразование датафрейма рейтингов в матрицу пользователь-элемент с заменой отсутствующих значений на нули.
        self.trainset = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)
        
        # Инициализация матриц для отклонений и количества оценок с размерностью "элемент-элемент".
        deviations = np.zeros((len(items), len(items)))
        counts = np.zeros((len(items), len(items)))
        
        # Создание индекса для маппинга ID фильмов в индексы матрицы.
        item_index = {item_id: index for index, item_id in enumerate(items)}
        
        # Обход всех рейтингов в датасете для вычисления отклонений и количеств.
        for row in ratings.itertuples():
            # Выборка рейтингов, поставленных текущим пользователем.
            user_items = self.trainset.loc[row.userId][self.trainset.loc[row.userId] > 0]
            for item_j in user_items.index:
                # Вычисление отклонения между парой фильмов и обновление счетчика оценок.
                i, j = item_index[row.movieId], item_index[item_j]
                deviations[i, j] += row.rating - user_items[item_j]
                counts[i, j] += 1
        
        # Конвертация матриц отклонений и количеств в датафреймы и сохранение в атрибуты класса.
        self.deviations = pd.DataFrame(np.divide(deviations, counts, where=counts!=0), index=items, columns=items)
        self.counts = pd.DataFrame(counts, index=items, columns=items)
    
    def predict(self, userId, itemId):
        # Фильтрация рейтингов пользователя для получения только тех элементов, которые оценены пользователем.
        user_ratings = self.trainset.loc[userId][self.trainset.loc[userId] > 0]
        
        # Отбор элементов, для которых можно вычислить прогноз на основе отклонений.
        relevant_items = user_ratings.index.intersection(self.deviations.index)
        
        # Исключение из рассмотрения самого элемента, для которого делается предсказание.
        relevant_items = relevant_items.difference([itemId])

        # Вычисление прогностического рейтинга на основе суммы взвешенных отклонений.
        if len(relevant_items) > 0:
            deviations_sum = sum([(self.deviations.loc[relevant_item, itemId] + user_ratings[relevant_item]) * self.counts.loc[relevant_item, itemId] for relevant_item in relevant_items])
            weights_sum = sum([self.counts.loc[relevant_item, itemId] for relevant_item in relevant_items if self.counts.loc[relevant_item, itemId] > 0])
            if weights_sum > 0:
                prediction = deviations_sum / weights_sum
            else:
                # Если нет данных для вычисления взвешенной суммы, используется средний рейтинг пользователя.
                prediction = user_ratings.mean()
        else:
            # Если у пользователя нет рейтингов для сравнения, используется глобальный средний рейтинг.
            prediction = self.trainset[self.trainset > 0].mean().mean()
        
        # Корректировка прогноза в допустимый диапазон оценок.
        prediction = max(0.5, min(5.0, prediction))
        
        return prediction
    
    def test(self, testset):
        # Получение предсказаний для набора данных теста.
        predictions = [self.predict(uid, iid) for uid, iid in testset]
        # Формирование и возврат списка результата с предсказаниями.
        return [(uid, iid, pred) for (uid, iid), pred in zip(testset, predictions)]

In [31]:
# Создаем экземпляр алгоритма SlopeOne.
algorithm = MySlopeOne()
# Обучаем модель на тренировочном наборе.
algorithm.fit(ratings[['userId', 'movieId', 'rating']]) 

# Создаем DataFrame из всех возможных пар "пользователь-элемент".
all_user_ids = ratings['userId'].unique()
all_movie_ids = ratings['movieId'].unique()
all_user_item_pairs = pd.MultiIndex.from_product([all_user_ids, all_movie_ids], names=['userId', 'movieId']).to_frame(index=False)

# Отфильтруем пары, которые существуют в DataFrame рейтингов
merged = all_user_item_pairs.merge(ratings, on=['userId', 'movieId'], how='left', indicator=True)
testset = merged[merged['_merge'] == 'left_only'][['userId', 'movieId']]

# Преобразуем в список кортежей, как ожидает метод MySlopeOne.test
testset_tuples = list(testset.itertuples(index=False, name=None))

# Получаем предсказания для отсутствующих значений.
predictions = algorithm.test(testset_tuples)

In [32]:
# Преобразуем список предсказаний в DataFrame.
pred_df = pd.DataFrame(predictions, columns=['userId', 'movieId', 'rating'])

# Объединяем предсказанные рейтинги с исходными рейтингами.
complete_ratings = pd.concat([ratings[['userId', 'movieId', 'rating']], pred_df])

# Создаем итоговую таблицу сводки с полными данными, включающую как фактические, так и предсказанные рейтинги.
complete_user_movie_rating = complete_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Сортируем столбцы итоговой сводной таблицы согласно исходной таблице сводки рейтингов.
complete_user_movie_rating = complete_user_movie_rating[user_movie_rating.columns]

# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,10,11,...,91529,91658,99114,106782,109487,112552,114060,115713,122882,131724
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.242424,4.000000,5.000000,3.242424,4.000000,4.242424,5.000000,5.000000,4.242424,...,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000
2,3.300000,4.500000,4.250000,5.000000,3.500000,4.750000,4.500000,5.000000,5.000000,4.500000,...,3.500000,2.500000,3.500000,5.000000,3.000000,4.000000,2.000000,3.500000,5.000000,5.000000
3,3.000000,0.500000,1.900000,0.833333,0.500000,2.200000,0.500000,0.833333,0.833333,0.500000,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.537975,2.777778,3.194444,3.777778,1.777778,3.569444,2.777778,3.777778,3.777778,2.777778,...,2.000000,3.000000,2.000000,0.500000,2.500000,1.500000,3.500000,2.000000,0.500000,0.500000
5,4.000000,3.277778,2.775510,4.277778,2.277778,3.510204,3.277778,4.277778,4.277778,3.277778,...,2.500000,3.500000,2.500000,1.000000,3.000000,2.000000,4.000000,2.500000,1.000000,1.000000
6,3.656627,4.000000,5.000000,3.000000,5.000000,4.000000,4.000000,3.000000,3.000000,4.000000,...,5.000000,5.000000,5.000000,3.500000,5.000000,4.500000,5.000000,5.000000,3.500000,3.500000
7,4.500000,3.750000,3.862500,4.750000,2.750000,4.212500,3.750000,4.750000,4.750000,3.750000,...,3.250000,4.250000,3.250000,1.750000,3.750000,2.750000,4.750000,3.250000,1.750000,1.750000


In [33]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")